Things that need to be done:
* create IAM role for boto, 

You now want to give boto account the following permisions:
* full ec2 creation access - or just enough to spin it up and run ssm commands on it
* create a policy that allows the boto account to assign a ssm role to ec2 instance

Create a role to assign ec2 instances:
* this role should give the ec2 instance permision to access the ssm API



NOTE: 
* You need your RTC time to be synched to the amazon UTC time
* Your local time can be set to your local time (GTM zone)

# Start Boto

In [5]:
import boto3 as bt3

In [6]:
boto3 = bt3.Session(profile_name='personal',region_name='eu-west-1')

In [7]:
s3 = boto3.resource('s3')

In [8]:
for bucket in s3.buckets.all():
    print(bucket.name)

jeroens-bucket
lauren-backups


# Generate Query

In [8]:
# generate date ranges
import datetime

#start = datetime.datetime.strptime("01-01-2016", "%d-%m-%Y")
start = datetime.datetime.strptime("28-11-2017", "%d-%m-%Y")
end = datetime.datetime.strptime("30-12-2017", "%d-%m-%Y")
dates_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

In [9]:
dates_generated

[datetime.datetime(2017, 11, 28, 0, 0),
 datetime.datetime(2017, 11, 29, 0, 0),
 datetime.datetime(2017, 11, 30, 0, 0),
 datetime.datetime(2017, 12, 1, 0, 0),
 datetime.datetime(2017, 12, 2, 0, 0),
 datetime.datetime(2017, 12, 3, 0, 0),
 datetime.datetime(2017, 12, 4, 0, 0),
 datetime.datetime(2017, 12, 5, 0, 0),
 datetime.datetime(2017, 12, 6, 0, 0),
 datetime.datetime(2017, 12, 7, 0, 0),
 datetime.datetime(2017, 12, 8, 0, 0),
 datetime.datetime(2017, 12, 9, 0, 0),
 datetime.datetime(2017, 12, 10, 0, 0),
 datetime.datetime(2017, 12, 11, 0, 0),
 datetime.datetime(2017, 12, 12, 0, 0),
 datetime.datetime(2017, 12, 13, 0, 0),
 datetime.datetime(2017, 12, 14, 0, 0),
 datetime.datetime(2017, 12, 15, 0, 0),
 datetime.datetime(2017, 12, 16, 0, 0),
 datetime.datetime(2017, 12, 17, 0, 0),
 datetime.datetime(2017, 12, 18, 0, 0),
 datetime.datetime(2017, 12, 19, 0, 0),
 datetime.datetime(2017, 12, 20, 0, 0),
 datetime.datetime(2017, 12, 21, 0, 0),
 datetime.datetime(2017, 12, 22, 0, 0),
 datetime

In [10]:
len(dates_generated)

32

In [11]:
# shuffle the dates around so that we dont have monotoicly increasing dates per instance
from random import shuffle
shuffle(dates_generated)
dates_generated

[datetime.datetime(2017, 12, 24, 0, 0),
 datetime.datetime(2017, 12, 26, 0, 0),
 datetime.datetime(2017, 11, 29, 0, 0),
 datetime.datetime(2017, 12, 6, 0, 0),
 datetime.datetime(2017, 12, 20, 0, 0),
 datetime.datetime(2017, 12, 12, 0, 0),
 datetime.datetime(2017, 12, 16, 0, 0),
 datetime.datetime(2017, 12, 27, 0, 0),
 datetime.datetime(2017, 12, 10, 0, 0),
 datetime.datetime(2017, 12, 17, 0, 0),
 datetime.datetime(2017, 12, 25, 0, 0),
 datetime.datetime(2017, 12, 3, 0, 0),
 datetime.datetime(2017, 12, 22, 0, 0),
 datetime.datetime(2017, 12, 23, 0, 0),
 datetime.datetime(2017, 11, 28, 0, 0),
 datetime.datetime(2017, 12, 14, 0, 0),
 datetime.datetime(2017, 12, 8, 0, 0),
 datetime.datetime(2017, 12, 28, 0, 0),
 datetime.datetime(2017, 12, 7, 0, 0),
 datetime.datetime(2017, 12, 9, 0, 0),
 datetime.datetime(2017, 12, 1, 0, 0),
 datetime.datetime(2017, 12, 18, 0, 0),
 datetime.datetime(2017, 12, 4, 0, 0),
 datetime.datetime(2017, 12, 11, 0, 0),
 datetime.datetime(2017, 12, 19, 0, 0),
 dateti

In [14]:
# generate dates that need to be queried

date_ranges_str = []
counter = 0
query = ''

#days_per_node = 41
days_per_node = 2


for date in dates_generated:
    counter += 1
    if counter <= days_per_node-1:
        query = query + date.strftime("%Y-%m-%d") + ";"
    elif counter == days_per_node:
        query = query + date.strftime("%Y-%m-%d")
        date_ranges_str.append(query)
        counter = 0
        query = ''#date.strftime("%Y-%m-%d") + ";"

if counter <= days_per_node:        
    date_ranges_str.append(query[0:-2])

In [15]:
days_per_node*80/60

2.6666666666666665

In [16]:
query

''

In [33]:
# generate complete query for eth

eth_queries = []
for date_range_q in date_ranges_str:
    query = 'su ec2-user -l -c "python ~/GetOldTweets-python/crypto.py --querysearch \'ethereum\' --days ' + "\'"+date_range_q+'\'"'
    eth_queries.append(query)


In [34]:
eth_nodes_needed = len(eth_queries)
eth_nodes_needed

17

In [35]:
# generate complete query for eth

bitcoin_queries = []
for date_range_q in date_ranges_str:
    query = 'su ec2-user -l -c "python ~/GetOldTweets-python/crypto.py --querysearch \'bitcoin\' --maxTweets \'150000\' --days ' + "\'"+date_range_q+'\'"'
    bitcoin_queries.append(query)


In [36]:
btc_nodes_needed = len(bitcoin_queries)
btc_nodes_needed

17

# Define bash execution Function

[docs on send_command paramaters](https://docs.aws.amazon.com/systems-manager/latest/userguide/sysman-paramstore-versions.html)

In [21]:
def execute_commands_on_linux_instances(client, commands, instance_ids,output_logs='boto-logs'):
    """Runs commands on remote linux instances
    :param client: a boto/boto3 ssm client
    :param commands: a list of strings, each one a command to execute on the instances
    :param instance_ids: a list of instance_id strings, of the instances on which to execute the command
    :return: the response from the send_command function (check the boto3 docs for ssm client.send_command() )
    """
    
    # allows the command to run for 3 days
    run_time = 60*60*24*3
    
    # execution time - max 2 houts
    execution_time = "172800"
    
    resp = client.send_command(
        DocumentName="AWS-RunShellScript", # One of AWS' preconfigured documents
        Parameters={'commands': commands,
                    'executionTimeout':[execution_time]},
        TimeoutSeconds=run_time,
        InstanceIds=instance_ids,
        OutputS3BucketName='jeroens-bucket',
        OutputS3KeyPrefix=output_logs
    )
    return resp

# Define Ec2

In [22]:
ec2 = boto3.client('ec2')

In [23]:
IAM_profile = {
        'Arn': 'arn:aws:iam::937520633893:instance-profile/EC2=S3+SSM'
    }



ami='ami-74e8eb9e'
key='jeroen-R-studio'
subnet='subnet-17a89c4f'
security_group=['sg-1c540361']

instance_market_options={
        'MarketType': 'spot',
        'SpotOptions': {
            'MaxPrice': '0.006',
            'SpotInstanceType': 'one-time',
            'InstanceInterruptionBehavior': 'terminate'
        }
}

# Spin up Ec2 instances for Eth

* if you want to run an isntance with an IAM role - the boto credentials need to have some admin rights
https://stackoverflow.com/questions/27266400/launching-ec-instance-a-client-error-unauthorizedoperation-occurred-when-ca

For a more in depth explination:
* https://aws.amazon.com/blogs/security/granting-permission-to-launch-ec2-instances-with-iam-roles-passrole-permission/

In [20]:
tags = [
                {
                    'ResourceType':'instance',
                    'Tags': [
                        {
                            'Key': 'use',
                            'Value': 'eth'
                        },
                    ]
                }]

In [21]:
eth_instances = ec2.run_instances(
        ImageId=ami,
        MinCount=eth_nodes_needed,
        MaxCount=eth_nodes_needed,
        KeyName=key,
        InstanceType='t1.micro',
        SubnetId=subnet,
        TagSpecifications=tags,
        IamInstanceProfile=IAM_profile,
        SecurityGroupIds=security_group,
        InstanceMarketOptions=instance_market_options,
        InstanceInitiatedShutdownBehavior='terminate'
        )

In [22]:
eth_instance_ids = []
for instance in eth_instances.get("Instances"):
    eth_instance_ids.append(instance.get("InstanceId"))
    
eth_instance_ids

['i-0b7dde412b20b300c',
 'i-0cb1f0579dfa994cf',
 'i-0cacb853dc90d018a',
 'i-071b4a815ce553bce',
 'i-085c7b377ec11de73',
 'i-05c073e808afba56d',
 'i-0e5c0361f7950c7e4',
 'i-068809c367e5d00a6',
 'i-09ef1b7fbfd021997',
 'i-0d95b5370da10d4ff',
 'i-0ec080b9987858e0a',
 'i-0503fc3e5410e8c4a',
 'i-07c8b55335ef73787',
 'i-00ce609e9bbe009b5',
 'i-0e071df7c736a4ad8',
 'i-0e02ba8b741d97887',
 'i-09cbf75163a1ba545',
 'i-012f5609cebbf713a',
 'i-0cd92477abc8bccf3',
 'i-0d51a5a8f7ff4cb2d',
 'i-0b2e149ea494754fe',
 'i-0fdb129cc134029ac']

## Run commands

In [23]:
eth_queries[-1]

'su ec2-user -l -c "python ~/GetOldTweets-python/crypto.py --querysearch \'ethereum\' --days \'2016-04-12;2018-05-18;2017-04-01;2018-02-22;2017-09-27;2016-06-09;2016-10-26;2016-04-09;2017-07-15;2017-05-19;2017-09-09;2017-04-27;2017-05-28;2016-03-23;2016-01-29;2018-05-06;2018-05-17;2018-02-09;2016-01-12;2017-07-2\'"'

In [24]:
ssm_client = boto3.client('ssm') 

eth_responseses = []

for instance_id, query in list(zip(eth_instance_ids,eth_queries)):
    print(instance_id)
    #print(type(query))
    responses = execute_commands_on_linux_instances(ssm_client, [query,"poweroff"], [instance_id],output_logs='boto-logs/eth/19-06-2018/')
    eth_responseses.append(responses)


i-0b7dde412b20b300c
i-0cb1f0579dfa994cf
i-0cacb853dc90d018a
i-071b4a815ce553bce
i-085c7b377ec11de73
i-05c073e808afba56d
i-0e5c0361f7950c7e4
i-068809c367e5d00a6
i-09ef1b7fbfd021997
i-0d95b5370da10d4ff
i-0ec080b9987858e0a
i-0503fc3e5410e8c4a
i-07c8b55335ef73787
i-00ce609e9bbe009b5
i-0e071df7c736a4ad8
i-0e02ba8b741d97887
i-09cbf75163a1ba545
i-012f5609cebbf713a
i-0cd92477abc8bccf3
i-0d51a5a8f7ff4cb2d
i-0b2e149ea494754fe
i-0fdb129cc134029ac


In [25]:
# list command-ids
for response in eth_responseses:
    command_id = response["Command"]["CommandId"]
    print(command_id)

f981afbd-cc8b-4a6a-acc5-2381267a1d3c
7689da4c-e61b-4757-b67c-4f0f54573fad
1bc31538-cfa0-4ca6-96f8-b9af682757b4
6d620281-4d33-4246-9b03-b3e37abc3b0e
cc8ce4bb-32a2-41da-aad2-edfb0e7d0d4c
b73c24be-2ae1-4e7b-85b9-60a0c36b076d
e4cdf45b-351f-4ee0-89bb-c888a4d1e3ba
4dac30ab-925c-418b-bd58-1158ba1616d8
cbda838e-cf93-42f6-90b6-dab126c7b130
f67e7401-e9cb-436c-938d-f9eaedaf6394
ecc1eaea-adb3-4db7-b3b6-13bf383511bb
3cb3230c-ecb3-43a4-97ea-b843d0719888
f6057bb6-339a-44b6-bacd-0fcc1fb7fbfe
a60dea8d-da6f-4ef8-80f9-9ccfea73d58e
2c3d527a-0555-4a3d-abeb-ed6adf503ec3
20d65f82-fe7a-4a35-bca0-8a4d609e1434
050d9655-f9e8-4015-b1bb-5db3bbe76ef4
d2131b06-49d1-40cb-a001-11549c4c0455
ee192ee8-6e11-4ead-a2f0-296b34a7b5ae
1a5665d5-139f-4e62-9840-d9ecd09a5483
d016df1c-f1aa-40eb-90af-ffddab9f3693
23d2012a-673b-4a42-9e42-088dbc5139bf


In [94]:
# kill commands
kill_eth_commands = False

if kill_eth_commands == True:
    for response in eth_responseses:
        command_id = response["Command"]["CommandId"]
        ssm_client.cancel_command(CommandId=command_id)

# Spin up Ec2 instances for Bitcoin

In [26]:
tags = [
                {
                    'ResourceType':'instance',
                    'Tags': [
                        {
                            'Key': 'use',
                            'Value': 'btc'
                        },
                    ]
                }]

In [27]:
ec2 = boto3.client('ec2')

In [28]:
btc_instances = ec2.run_instances(
        ImageId=ami,
        MinCount=btc_nodes_needed,
        MaxCount=btc_nodes_needed,
        KeyName=key,
        InstanceType='t1.micro',
        SubnetId=subnet,
        TagSpecifications=tags,
        IamInstanceProfile=IAM_profile,
        SecurityGroupIds=security_group,
        InstanceMarketOptions=instance_market_options,
        InstanceInitiatedShutdownBehavior='terminate'
        )

In [29]:
instance_ids = []
for instance in btc_instances.get("Instances"):
    instance_ids.append(instance.get("InstanceId"))
    
instance_ids

['i-002253254abc15890',
 'i-09eb9ebf0593fc2ad',
 'i-0a5142385c6d2419a',
 'i-0cfa1d5a7628bf268',
 'i-0711a8c0ce2c53920',
 'i-044b196abc46dc3bf',
 'i-0f78f580ff8080e15',
 'i-00c8044e77fa370a4',
 'i-069cdf5d1489af393',
 'i-0bb3cbb03af6d5eb5',
 'i-0b6f6a918df29bf0e',
 'i-0a152486d56d52e58',
 'i-07842b6b52c5685c2',
 'i-0c068603db5d95a0b',
 'i-034717ab0ef0f2973',
 'i-0f343398682685d11',
 'i-0d9e0c3c94a67a896']

## Run commands

In [38]:
bitcoin_queries[0]

'su ec2-user -l -c "python ~/GetOldTweets-python/crypto.py --querysearch \'bitcoin\' --maxTweets \'150000\' --days \'2017-12-24;2017-12-26\'"'

In [39]:
ssm_client = boto3.client('ssm') # Need your credentials here

btc_responseses = []

for instance_id, query in list(zip(instance_ids,bitcoin_queries)):
    print(instance_id)
    #print(type(query))
    btc_responses = execute_commands_on_linux_instances(ssm_client, [query,"poweroff"], [instance_id],output_logs='boto-logs/btc/22-06-2018/')
    btc_responseses.append(btc_responses)


i-002253254abc15890
i-09eb9ebf0593fc2ad
i-0a5142385c6d2419a
i-0cfa1d5a7628bf268
i-0711a8c0ce2c53920
i-044b196abc46dc3bf
i-0f78f580ff8080e15
i-00c8044e77fa370a4
i-069cdf5d1489af393
i-0bb3cbb03af6d5eb5
i-0b6f6a918df29bf0e
i-0a152486d56d52e58
i-07842b6b52c5685c2
i-0c068603db5d95a0b
i-034717ab0ef0f2973
i-0f343398682685d11
i-0d9e0c3c94a67a896


In [40]:
len(btc_responseses)

17

In [41]:
# list command-ids
for response in btc_responseses:
    command_id = response["Command"]["CommandId"]
    print(command_id)

9f718711-907a-41d9-a029-526c3c85def9
f5a7fb03-59f4-418d-85e5-33a1071a699e
b27a04ef-94d1-4730-9044-e8b12e31eac2
a3884743-8b83-4765-a192-2ed0f1b57639
a0e326c0-5ca8-42f3-a36f-54a2945676b3
c6c65dd4-38e3-4c1c-93d4-92250550ab1f
4eaa429c-e382-47d3-80db-30839a0da584
a65a9378-0c02-4858-b7df-f6ec0d44f60d
9a37cc7d-b07e-4346-a2a3-65060bd604b8
0bfc9ad3-ea9e-4fed-a648-fd412c55c0d9
0d8fe836-0f20-41e6-84b6-4374c3028e16
ba2a838b-5c62-4cf5-aacb-da05f7831cea
e6962474-3b11-44a9-b3e0-35e2dc4c75d6
09ad3148-3c46-4b5b-972e-51ae9ea2e47d
b0485773-8adb-49cb-94e0-b12731737426
cfb7e7a1-33f1-4005-adfa-87a38e9f401f
26476d7d-6d74-4ef9-8514-9c88e1a4b08d


In [ ]:
# kill commands
kill_eth_commands = False

if kill_eth_commands == True:
    for response in btc_responseses:
        command_id = response["Command"]["CommandId"]
        ssm_client.cancel_command(CommandId=command_id)

# Kill All SSM commands

In [9]:
ssm_client = boto3.client('ssm') # Need your credentials here
active_ssm = ssm_client.list_commands(Filters=[
        {
            'key': 'Status',
            'value': 'InProgress'
        }])

In [10]:
active_ssm

{'Commands': [{'CommandId': 'e6962474-3b11-44a9-b3e0-35e2dc4c75d6',
   'DocumentName': 'AWS-RunShellScript',
   'DocumentVersion': '',
   'Comment': '',
   'ExpiresAfter': datetime.datetime(2018, 6, 27, 7, 44, 53, 853000, tzinfo=tzlocal()),
   'Parameters': {'commands': ['su ec2-user -l -c "python ~/GetOldTweets-python/crypto.py --querysearch \'bitcoin\' --maxTweets \'150000\' --days \'2017-12-19;2017-12-13\'"',
     'poweroff'],
    'executionTimeout': ['172800']},
   'InstanceIds': ['i-07842b6b52c5685c2'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2018, 6, 22, 7, 44, 53, 853000, tzinfo=tzlocal()),
   'Status': 'InProgress',
   'StatusDetails': 'InProgress',
   'OutputS3BucketName': 'jeroens-bucket',
   'OutputS3KeyPrefix': 'boto-logs/btc/22-06-2018/',
   'MaxConcurrency': '50',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 0,
   'ErrorCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    

In [39]:
active_ssm["Commands"][0]["CommandId"]

'1e864287-af28-46a8-a0f0-f87ff274924f'

In [11]:
for response in active_ssm["Commands"]:
    command_id = response["CommandId"]
    print(command_id)
    ssm_client.cancel_command(CommandId=command_id)

e6962474-3b11-44a9-b3e0-35e2dc4c75d6
ba2a838b-5c62-4cf5-aacb-da05f7831cea
0bfc9ad3-ea9e-4fed-a648-fd412c55c0d9
a65a9378-0c02-4858-b7df-f6ec0d44f60d
c6c65dd4-38e3-4c1c-93d4-92250550ab1f
f5a7fb03-59f4-418d-85e5-33a1071a699e
9f718711-907a-41d9-a029-526c3c85def9
9cb8654c-edf5-4307-b9d5-7a82f1a0afe8
68503298-2d78-466d-8941-0e33fa03b104
98c89797-ae55-4062-ba37-cfbc95de9413
ae0762da-ba13-4d82-b4df-85aee142776f
696e153e-3f11-4e0b-a53a-0be8a368738b
6de7f919-2a1c-458a-9a7c-79acdd951561
7511ecb1-f195-48f9-8548-f7e1323dc819
4a267221-4fff-44ac-b44e-effe083a7fc5
650ec8ed-4289-45c9-aeba-c4eb1d7b8699
fc3b952a-1097-4eb3-ac53-2ab3ffdb9e17
b7c9eb03-8807-4e08-aab8-a714888e94d8
23d2012a-673b-4a42-9e42-088dbc5139bf
1a5665d5-139f-4e62-9840-d9ecd09a5483
d2131b06-49d1-40cb-a001-11549c4c0455
050d9655-f9e8-4015-b1bb-5db3bbe76ef4
20d65f82-fe7a-4a35-bca0-8a4d609e1434
f6057bb6-339a-44b6-bacd-0fcc1fb7fbfe
3cb3230c-ecb3-43a4-97ea-b843d0719888
ecc1eaea-adb3-4db7-b3b6-13bf383511bb
4dac30ab-925c-418b-bd58-1158ba1616d8
6

# Commands

In [ ]:
# set up proccess
commands = [
'yum update -y',
'yum install git -y',
'wget https://repo.continuum.io/miniconda/Miniconda2-latest-Linux-x86_64.sh',
'chmod 777 Miniconda2-latest-Linux-x86_64.sh',
'bash ./Miniconda2-latest-Linux-x86_64.sh -b',
'echo "export PATH=/root/miniconda2/bin:$PATH" >> /root/.bashrc ',
'conda install boto -y',
'sudo yum groupinstall "Development Tools" -y',
'yum install python-lxml',
'yum install libxml2-devel libxslt-devel python-devel',
'git clone https://github.com/Jefferson-Henrique/GetOldTweets-python.git',
'pip install -r requirements.txt',
'mkdir /GetOldTweets-python/Bitcoin',
'mkdir /GetOldTweets-python/Ethereum',
'aws s3 cp s3://jeroens-bucket/cryto_v2.py GetOldTweets-python/cryto_v2.py'
]